In [1]:
from MMT_reduce_module import *

In [2]:
from __future__ import print_function
import numpy as np
import scipy as sp
import scipy.ndimage
from astropy.io import fits
import sys
import numpy.ma as ma
import math
import pidly
import os
import glob
import time
import matplotlib.pyplot as plt

from shift_methods import *

%matplotlib inline
%load_ext autoreload
%autoreload 2

from astropy.visualization import ZScaleInterval
interval = ZScaleInterval()

## MMT_reduce_module Testing

#### The purpose of this notebook is to test functions in the MMT module one-by-one and examine the outputs at each step.

Major to-do items (Updated 7/6/2019): 

* Fix saturated data!
    * Examine how sky frames are being made -- **done, 7/5**
    * Examine how sky subtraction is being performed on science data -- **done, 7/5**
    * Determine best practice of aligning saturated frames -- **done, 7/6**
    * plt.imshow reduced steps -- **done, sometime**
    * Stacking size issue -- **done**
    * Update placeholder star position in saturated images from 512, 512 (before measuring companion positions)  
    
    
* Determine how flatfield generation is being done and improve if need be
    * Debug negative dark frames/negative flat frame issues? -- **done, 5/23**
    * Double-check dome flat correction, now that twilight flat correction is working (with correct inputs)
    
More to-do items (last updated 7/5):
* Ensure corquad correction is applied prior to analysis steps - **done, 5/23**
* Define location for reduced data to be written -- **done, 7/1**
* Check to ensure all flatfields are taken in the same filter - update within module -- **done, 7/1**
* Rotation angle correction and image flip for MMT data (appears to be same VLT conventions) - update within module and here -- **done, 7/5**
* Make sure align flag opens ds9 with frames opening in correct order -- **done, 7/5**
* Cross-check sky subtraction in case of very saturated data, and if small changes in rotation angle make difference

## Cell #1 to Update: Set up paths to raw data and definitions

In [13]:
# Always update and double-check the following seven paths and variables:

path_to_raw_sci = '/Users/u5ajain/Dropbox (Amherst College)/2016_10_11_run/20161012/HIP_Targets_20161012/HIP15220/30.0_copy/'
path_to_raw_darks = '/Users/u5ajain/Dropbox (Amherst College)/2016_10_11_run/Calibrations/darks/'
path_to_raw_flats = '/Users/u5ajain/Dropbox (Amherst College)/2016_10_11_run/Calibrations/twilight/sky flat/20.0/'

objname = 'HIP15220_30s'

flattype = 0 # 0 for sky, 1 for lamp
saturated = 1 # 0 if unsaturated, 1 if saturated
alignflag = 0 # 0 if single star or faint companion, 1 if equal brightness binary or difficult to align for some reason

## Cell #2 to Update: Define the x-y position of the saturated star from the first "qim" image

In [10]:
# Update the following with your estimate from ds9
ref_imagex = 493
ref_imagey =  395

# No need to edit these two lines
ref_imagex = 1024 - ref_imagex
imsize = 1024

## The following cells are each of the steps of the reduce_raw_sci function

In [ ]:
# Make list of science frames and check exposure time 
scilist = glob.glob(path_to_raw_sci + 'q*.fits')

scilist.sort()

print(f"Number of science frames found: {len(scilist)} \n")

In [ ]:
scitimes = [fits.getheader(im, ignore_missing_end = True)['EXPTIME'] for im in scilist]

# check if all of the exposure times in the current directory are the same:
if all(x == scitimes[0] for x in scitimes):
    print("Science frame exposure time: " + str(scitimes[0]) + "\n")
else:
    raise Exception("Exposure times for given list of files do not match. \
    You may need to make/define separate subfolders for different exptimes.")

sci_exptime = scitimes[0]

n = len(scilist)


# get header from science frames to work with
sciheader = fits.getheader(scilist[0])


In [ ]:
# check for datacubes
if len(fits.getdata(scilist[0]).shape) == 3: # check for data cubes of science frames
    sciarray = np.zeros([imsize,imsize,n*fits.getdata(scilist[0]).shape[0]])
else:
    sciarray = np.zeros([imsize,imsize,n])

if len(fits.getdata(scilist[0]).shape) == 3: # check for data cubes of science frames    
    totalframes = n*fits.getdata(scilist[0]).shape[0]
else:
    totalframes = n

In [ ]:
# in case data were rotated during observing sequence, set up empty array of rotation angles
angle = np.zeros(totalframes)

im_index = 0


for ii in range(0, n):
    im = fits.getdata(scilist[ii], ignore_missing_end=True)
    header = fits.getheader(scilist[ii],ignore_missing_end=True)

    # flip image left-right, as required for MMT data:
    im = np.fliplr(im)

    if len(im.shape) == 3: # check for data cubes of science frames
        assert not np.any(np.isnan(im))
        for jj in range(0, im.shape[0]):
            sciarray[:,:,im_index] = im[jj,:,:]
            angle[im_index] = (header['PA'] - header['ROT']) * (np.pi/180.0)
            im_index += 1
    else: 
        sciarray[:,:,ii] = im  
        angle[ii] = (header['PA'] - header['ROT']) * (np.pi/180.0)
    header = fits.getheader(scilist[ii], ignore_missing_end=True)    





In [ ]:
print(angle)
print(im)

In [ ]:
#MASTER_DARK

print("Creating and applying master darks and flats...\n")    

# create master dark matching science exposure times
med_dark = dark_combine(path_to_raw_sci, path_to_raw_darks, sci_exptime, imsize, objname) 

vmin, vmax = interval.get_limits(med_dark)
plt.imshow(med_dark, vmin=vmin, vmax=vmax, origin='lower')

In [ ]:
# subtract off the median dark frame from each of the science frames
for ii in range (0, totalframes):
    sciarray[:,:,ii] -= med_dark


In [ ]:
flatlist = glob.glob(path_to_raw_flats + '**/q*fits',recursive = True)

In [ ]:
#MEDIAN_AND_MASTER_FLAT

# create the masterflat 
med_flat, master_flat, flatheader = process_flats(path_to_raw_sci, path_to_raw_flats, path_to_raw_darks, imsize, flattype, objname)

vmin, vmax = interval.get_limits(med_flat)
plt.imshow(med_flat, vmin=vmin, vmax=vmax, origin='lower')

In [ ]:
# divide each science frame by the masterflat frame
for ii in range(0, totalframes):
    sciarray[:,:,ii] /= master_flat



In [ ]:
#BAD_PIXEL_MAP

print("Creating bad pixel map and correcting for bad pixels and cosmic rays. \n",
     "This may take a moment... \n") 

# create bad pixel map
badflat = badpixelmap(path_to_raw_sci, med_flat, objname, flatheader)  

vmin, vmax = interval.get_limits(badflat)
plt.imshow(badflat, vmin=vmin, vmax=vmax, origin='lower')

In [ ]:
# correct the bad pixels and cosmic rays
reduced_sciarray = correct_bad_pixels(sciarray, badflat)


In [ ]:
# write out a test reduced science image 
fits.writeto(path_to_raw_sci+'test_reduced_science.fits', reduced_sciarray[:,:,0], overwrite=True)

In [ ]:
#MASTER_SKY
print("Creating master sky from science frames...\n") 

# create median sky from stack of science images
sky_output = create_sky_frames(path_to_raw_sci, reduced_sciarray, sciheader, objname, angle)



In [ ]:
# get median and examine sky output:
print(np.median(sky_output[1]))
vmin, vmax = interval.get_limits(sky_output[1])
plt.imshow(sky_output[1], vmin=vmin, vmax=vmax, origin='lower')

In [ ]:
# apply sky subtraction to each science image 
skysub_science_array, rot_flag = sky_subtract(reduced_sciarray, sky_output, angle)

In [ ]:
#INDIVIDUAL_REDUCED_SCI_IMAGES
t0=time.time()

# initialize blank list to hold all of the reduced science image names
scinames_list = []


for ii in range(0, totalframes):
    print(f"Saving reduced frame #{ii}")
    sciname = 'reducedsci_00' + str(ii) + '.fits'
    if ii >= 10:
        sciname = 'reducedsci_0' + str(ii) + '.fits'
    if ii >= 100:
        sciname = 'reducedsci_' + str(ii) + '.fits'
    fits.writeto(path_to_raw_sci+sciname, skysub_science_array[:,:,ii], sciheader, overwrite = True, output_verify='silentfix')
    scinames_list.append(sciname)
    
t1=time.time()
print("Time taken: ", (t1-t0)/60.)

## Measure Star Centers

In [ ]:
# get directory where reduced frames are written
current_dir = path_to_raw_sci

# measure star positions in all of the images
if saturated == 0:
    xcen, ycen = measure_star_centers(path_to_raw_sci, skysub_science_array, scinames_list, sciheader, saturated, alignflag, current_dir, saveframes = True)

elif saturated == 1:    
    xcen, ycen = cross_correlate_centers(path_to_raw_sci, skysub_science_array, scinames_list, ref_imagex, ref_imagey)    

else:
    raiseException("Saturated flag not recognized.")

In [ ]:
print('xcen:', xcen, 'ycen:', ycen)

In [ ]:
#SHIFTED_IMAGES
#STACKED
#FINAL

# get current directory where reduced frames are written
current_dir = path_to_raw_sci

t0=time.time()
# final step (!) - shift and combine all of the images.
rotate_shift_align(path_to_raw_sci, xcen, ycen, angle, skysub_science_array, objname, sciheader, current_dir, imsize=1024)

t1 = time.time()
timetaken = (t1-t0)/60.
print(f"Completed reduction of {totalframes} images in {timetaken} minutes.")








## Run all of the above reduction steps in one go:

In [14]:
#Running all the above reduction steps in one go
reduce_raw_sci(path_to_raw_sci, path_to_raw_darks, path_to_raw_flats, objname, flattype, saturated, alignflag, ref_imagex, ref_imagey, imsize = 1024)

Number of science frames found: 20 

Science frame exposure time: 30.0

Creating and applying master darks and flats...

Found 16 darks with exposure times of 30.0. 

Found 30 total flats taken in Ks band filter.

Found 30 twilight flats with exposure times of 20.0. 

Creating new master dark for flats with 20.0s exposure. 

Found 5 darks with exposure times of 20.0. 

Creating bad pixel map and correcting for bad pixels and cosmic rays. 
 This may take a moment... 

Correcting image #0
Correcting image #1
Correcting image #2
Correcting image #3
Correcting image #4
Correcting image #5
Correcting image #6
Correcting image #7
Correcting image #8
Correcting image #9
Correcting image #10
Correcting image #11
Correcting image #12
Correcting image #13
Correcting image #14
Correcting image #15
Correcting image #16
Correcting image #17
Correcting image #18
Correcting image #19
Creating master sky from science frames...

Saving reduced frame #0
Saving reduced frame #1
Saving reduced frame #2
Sa